
# Analyzing Animal Movement Patterns: Unveiling Behavior Insights

**Introduction**
The primary objective of this analysis was to uncover meaningful patterns from complex movement trajectories that could offer deeper insights into animal behavior.

**Methodology: Encoding Movement into Meaningful Patterns**
*String-Based Representation*: To transform the raw movement data into a structured format, a method was devised to encode the animals' movement trajectories into string-based representations. This systematic approach captured the directional components of the animals' movements, forming the basis for subsequent analyses.

**Identifying Individual Animal Behavior Patterns**
*Pattern Recognition Techniques*: Techniques were employed to identify frequent patterns within the movement strings for each individual animal. By meticulously extracting and categorizing these recurring sequences, distinct behavioral tendencies specific to each animal were discerned.

**Unveiling Cross-Species Behavioral Trends**
*Comparative Analysis of Frequent Patterns*: Moving beyond individual animal behaviors, the analysis delved into the realm of inter-animal interactions. By comparing the frequent patterns identified across various species, common behavioral tendencies that transcended species boundaries were unveiled.

**Visual Representation for Enhanced Insights**
*Data Visualization Techniques*: The culmination of the analysis was presented through visualizations that effectively communicated the identified frequent movement patterns and cross-species behavioral trends. These visual representations not only enhanced understanding but also served as a valuable resource for further interpretation.

**Conclusion**
The analysis provided a systematic approach to extract insights from raw movement data. By deciphering frequent patterns within individual animal behaviors and uncovering shared tendencies across species, this work contributed to a comprehensive understanding of common behavioral traits. The visualizations generated from this analysis serve as a powerful tool for researchers, enabling them to navigate the complexities of animal movement behaviors and their interactions effectively.



## Importing and basic preprocessing


- The dataset of animal movement data is read from a CSV file.
- The data is processed by resampling it into hourly intervals for each unique animal identifier.
- The data is resampled using median values, with the omission of missing values, and the inclusion of the animal identifier.



In [30]:
import pandas as pd
import numpy as np
import plotly.express as px
import random
import plotly.graph_objects as go


In [57]:
df = pd.read_csv('data/BCI-movement-filtered.csv')
resampled_list = []
for animal in pd.unique(df['individual-local-identifier']):
    animal_df = df[df['individual-local-identifier'] == animal]

    # Extract the relevant columns and convert the timestamp to datetime
    danieldf = animal_df.loc[:, ['location-lat', 'location-long', 'study-local-timestamp']]
    danieldf['study-local-timestamp'] = pd.to_datetime(danieldf['study-local-timestamp'], format='mixed')

    # Resample the data to 1-hour intervals
    
#     resampled_data = danieldf.resample('1H', on='study-local-timestamp').median()
#     resampled_data.dropna(inplace=True)
#     resampled_data.reset_index(inplace=True)


    # Change the animal identifier to a variable
    resampled_data['individual-local-identifier'] = animal
    resampled_list.append(resampled_data)
resampled_data = pd.concat(resampled_list, ignore_index=True)
# resampled_data.columns = ['ts', 'lat', 'lon', 'id']

In [59]:
pqr=df.loc[:, ['location-lat', 'location-long', 'study-local-timestamp', 'individual-local-identifier']]
pqr.columns=[ 'Lat', 'Lon','Date/Time', 'ID']
pqr.to_csv('bandar.csv',index=False)

## Visualization
This code performs data visualization using the Plotly library to display animal movement patterns on a map.







In [36]:
#smalldf = resampled_data[resampled_data['id'] == 'Olenna Tyrell']
smalldfs = []
unique_animal_ids = resampled_data['id'].unique()

for animal_id in unique_animal_ids:
    smalldf = resampled_data[resampled_data['id'] == animal_id].copy()

    # Generate a color for this specific animal
    animal_color = '#' + ''.join([random.choice('0123456789ABCDEF') for _ in range(6)])
    
    # Assign the color to all rows of this animal's DataFrame
    smalldf['color'] = animal_color

    smalldfs.append(smalldf)



def plotting(smalldf, title):
    
    mapboxtoken="pk.eyJ1IjoiYWJyb2xzbSIsImEiOiJjbGt5aHhmMDYwMnBxM25vMWRrdHl3Z2E3In0.NzmidKIzpJRKKYXyLcXwhw"

    fig = go.Figure()

    fig.add_trace(
        go.Scattermapbox(
            lat=smalldf['lat'],
            lon=smalldf['lon'],
#             mode='markers+lines+text',
            mode='markers+text',
            marker=dict(size=10, color=smalldf['color']),
            text= smalldf['ts'].dt.strftime('%D %H:%M') + ' ' + smalldf['id'],
            hoverinfo='text',
             line=dict(color='grey',width=1),
            textfont=dict(color='white', size=14),
        )
    )

        # Update layout for the plot
    fig.update_layout(
            title=f'{title}',
            xaxis_title='UTM Easting',
            yaxis_title='UTM Northing',
            hovermode='closest',
            mapbox=dict(
                accesstoken=mapboxtoken,
                style='satellite',
                bearing=0,
                center=dict(lat=smalldf['lat'].mean(), lon=smalldf['lon'].mean()),
                zoom=16
            ),
            width=1980,
            height=1024
        )

        # Show the plot
    fig.show()
    
for small_v in smalldfs:
    1
#     plotting(small_v, 'Daniel')
#     break



SyntaxError: incomplete input (3832197793.py, line 62)


## `get_day_times_panama` Function

This function assigns a segment label ('M', 'D', 'E', 'N', 'F') to a given time in the Panama timezone. It categorizes the input time into segments: morning, day, evening, night, or undefined ('F') based on predefined time ranges. 

### Input:
- `current_time_str` (str): A time string in the format '%H:%M' representing the current time.

### Output:
Returns a string representing the segment label associated with the input time.

## `calculate_distance` Function

This function calculates the distance between two GPS coordinates using the geopy library's `geodesic` function. It computes the distance in meters between two points on the Earth's surface.

### Input:
- `coord1` (tuple): A tuple containing latitude and longitude of the first point.
- `coord2` (tuple): A tuple containing latitude and longitude of the second point.

### Output:
Returns the distance in meters between the two given coordinates.

## `calculate_direction` Function

This function calculates the cardinal direction ('N', 'S', 'E', 'W') based on the relative positions of two GPS coordinates. It utilizes trigonometric calculations to determine the direction.

### Input:
- `lat1` (float): Latitude of the first point.
- `lon1` (float): Longitude of the first point.
- `lat2` (float): Latitude of the second point.
- `lon2` (float): Longitude of the second point.

### Output:
Returns a string representing the direction or a combination of directions ('NE', 'SE', 'SW', 'NW').

## `directional_mapping` Function

This function performs directional mapping and segment labeling on a DataFrame containing GPS data. It calculates distances and directions between consecutive points, assigns direction labels, and segments based on time.

### Input:
- `df` (pandas DataFrame): A DataFrame containing GPS data with columns 'ts' (timestamp), 'lat' (latitude), and 'lon' (longitude).

### Output:
Returns a tuple:
- A string concatenating all the direction labels.
- A modified DataFrame with added columns ('prev_lat', 'prev_lon', 'distance', 'direction', 'Segment').



In [37]:
#
from geopy.distance import geodesic
import math


from datetime import datetime, time, timedelta
## Integrate the date/time information while doing analysis 
def get_day_times_panama(current_time_str):
    # Convert the current time string to a datetime object
    current_time = datetime.strptime(current_time_str, '%H:%M').time()

    # Define time ranges for morning, rest, and night
    morning_start = time(6, 0)
    morning_end = time(11, 0)
    rest_start = time(16, 0)
    rest_end = time(19, 0)

    # Determine the segment for the given time
    if morning_start <= current_time < morning_end:
        return 'M'
    elif morning_end <= current_time <rest_start:
        return 'D'
    elif rest_start <= current_time < rest_end:
        return 'E'
    elif time(19, 0) <= current_time < time(23, 0):
        return 'N'
    elif time(0, 0) >= current_time < time(6, 0):
        return 'N'
    else:
        return 'F'

# Create a pandas DataFrame with a column containing time values


# Convert Timestamp to string and apply the function to the 'Time' column



# Function to calculate the distance between two GPS positions using geopy
def calculate_distance(coord1, coord2):
    return geodesic(coord1, coord2).meters

def calculate_direction(lat1, lon1, lat2, lon2):
    direction = ""
    if lat1 < lat2:
        direction += "N"
    elif lat1 > lat2:
        direction += "S"
        
    if lon1 < lon2:
        direction += "E"
    elif lon1 > lon2:
        direction += "W"
        
    return direction



def calculate_direction(lat1, lon1, lat2, lon2):
    # Convert latitude and longitude from degrees to radians
    lat1_rad = math.radians(lat1)
    lon1_rad = math.radians(lon1)
    lat2_rad = math.radians(lat2)
    lon2_rad = math.radians(lon2)

    # Calculate the differences in latitudes and longitudes
    dlat = lat2_rad - lat1_rad
    dlon = lon2_rad - lon1_rad

    # Calculate the angle using atan2 and convert it to degrees
    angle = math.degrees(math.atan2(dlat, dlon))

    # Normalize the angle to be within the range of [0, 360)
    angle = (angle + 360) % 360

    # Map the angle to the corresponding direction
    if 337.5 <= angle < 22.5:
        return 'W'
    elif 22.5 <= angle < 67.5:
        return 'SW'
    elif 67.5 <= angle < 112.5:
        return 'S'
    elif 112.5 <= angle < 157.5:
        return 'SE'
    elif 157.5 <= angle < 202.5:
        return 'E'
    elif 202.5 <= angle < 247.5:
        return 'NE'
    elif 247.5 <= angle < 292.5:
        return 'N'
    elif 292.5 <= angle < 337.5:
        return 'NW'
    else:
        return ''  # Invalid input or undefined direction

    
def directional_mapping(df):
    # Create a copy of the DataFrame to avoid modifying the original DataFrame
    df_copy = df.copy()

    # Rename the columns in the copied DataFrame
    #df_copy.columns = ['ts', 'lat', 'lon', 'id', 'color']

    # Rest of the code remains the same
    df_copy['prev_lat'] = df_copy['lat'].shift(1)
    df_copy['prev_lon'] = df_copy['lon'].shift(1)

    # Calculate the distance in meters from the previous ts
    df_copy['distance'] = df_copy.apply(lambda row: calculate_distance(
        (row['lat'], row['lon']),
        (row['prev_lat'], row['prev_lon'])
    ) if pd.notnull(row['prev_lat']) and pd.notnull(row['prev_lon']) and (row['ts'] - df_copy.loc[row.name - 1, 'ts']).seconds / 3600 <= 6 else float('nan'), axis=1)

    # Apply the direction label calculation to the DataFrame
    df_copy['direction'] = df_copy.apply(lambda row: calculate_direction(
        row['lat'], row['lon'], row['prev_lat'], row['prev_lon']), axis=1)

    # Drop the lag columns
    # df_copy.drop(columns=['prev_lat', 'prev_lon'], inplace=True)

    direction_mapping = {'NE': '1', 'SE': '2', 'SW': '3', 'NW': '4', 'N': 'N', 'S': 'S', 'W': 'W', 'E': 'E', '': ' '}
    df_copy['direction'] = df_copy['direction'].map(direction_mapping)
    df_copy['Segment'] = df_copy['ts'].dt.strftime('%H:%M').apply(lambda x: get_day_times_panama(x))

    #df_copy['Movement'] = df_copy['direction'] #+ df_copy['Segment']
    return ''.join(df_copy['direction']).replace(' ',''), df_copy

patterns = []
smalldfs_updated = []
for smalldf in smalldfs:
    pattern ,smalldf= directional_mapping(smalldf)
    patterns.append(pattern)
    smalldfs_updated.append(smalldf)
smalldfs = smalldfs_updated
print(len(patterns))

14


In [39]:
patterns[0]

'S2S11144E1SS3S3N114NEN2N444ES111111333NNN1E1S34S41E1E1SSS3N22NNNNE33N4EE3114NNENSS411E1NEE3N332S3E1NNEE2334N2SNEEE334N431N222NEN2N4NS3NS2213411ES321241EN411SSEEE1EEEE11S2EE3N4E1N3SE2SS2ES4N443S22EN11E1N2SN13114N14SNSS3421NN144332S1SS4214NNN3SSE1E4341ES42S3E4444NNEE11SSN1E4E14412NN2N11EE23244NS222EE4N3S4NN2EN1EE2422S343S3321NES312NN1EE422ES324N144S42EEEE32N321ENN1333NNE3ENN334NNE4SSSS34NN1NE14N2E33SNN31ENS2E3EE1E1342SE4S223111NN2N411S22SS114ES3S34EE1N4242SS334NNN12N3N4NE1EN2E43N4S3S2221NNS332ENNS2EE4E33341NN11N14ES32S22N1N4E4S2222EEEE14N41SS4E4E11442233342NN1SNN12S33S342E2E4N4144N4SSE223NE4S14N3S2S41E444S33224NSN12E1S1N33233332NENS3134ENN3S32S42E24414N1NESE2S443N3223SEEEES441SSS44411223'

## Frequent Patterns

## `find_frequent_patterns` Function

This function aims to identify frequent patterns within a given string, based on specified criteria. It operates by sliding a window over the string, generating patterns of varying lengths, and then counting their occurrences. The patterns are filtered to exclude those containing a specific character (in this case, `"."`). The function returns a list of frequent patterns sorted by a calculated metric.

### Input Parameters:

- `string` (str): The input string from which patterns are extracted and analyzed.
- `min_length` (int): The minimum length of patterns to consider.
- `max_length` (int): The maximum length of patterns to consider.
- `min_frequency` (int): The minimum count required for a pattern to be considered frequent.

### Output:

The function returns a list of tuples, each containing a frequent pattern and its corresponding count, meeting the specified criteria. The list is sorted in descending order based on a calculated metric that considers both pattern length and occurrence count.


In [46]:

from collections import defaultdict

def find_frequent_patterns(string, min_length, max_length, min_frequency):
    patterns = defaultdict(lambda: 0)

    for length in range(min_length, max_length + 1):
        for i in range(len(string) - length + 1):
            pattern = string[i:i+length]
            if '.' not in pattern:  # Skip patterns containing "."
                patterns[pattern] += 1

    frequent_patterns = [(pattern, count) for pattern, count in patterns.items() if count >= min_frequency]
    sorted_patterns = sorted(frequent_patterns, key=lambda item: len(item[0]) * item[1], reverse=True)

    return sorted_patterns


# Example string
frequent_patterns = []
for pattern in patterns:
    directional_string = pattern# "NORTHEASTNORTHWESTSOUTHEASTSOUTHWEST"

    min_length = 4
    max_length = 10
    min_frequency = 2

    frequent_pattern = find_frequent_patterns(directional_string, min_length, max_length, min_frequency)
    frequent_patterns.append(frequent_pattern)
#     print(frequent_pattern)
print(frequent_patterns[0])

[('334N', 4), ('EEEE', 4), ('114N', 3), ('N2N4', 3), ('1111', 3), ('4NNE', 3), ('ES32', 3), ('EEE1', 3), ('S222', 3), ('S332', 3), ('34NN', 3), ('NEN2N4', 2), ('1333NN', 2), ('NEN2N', 2), ('EN2N4', 2), ('11111', 2), ('1333N', 2), ('333NN', 2), ('3114N', 2), ('11E1N', 2), ('N411S', 2), ('EEEE1', 2), ('EE11S', 2), ('222EE', 2), ('2EEEE', 2), ('334NN', 2), ('E4S22', 2), ('14ES3', 2), ('S111', 2), ('1144', 2), ('E1SS', 2), ('S3S3', 2), ('NEN2', 2), ('EN2N', 2), ('1333', 2), ('333N', 2), ('33NN', 2), ('NNN1', 2), ('NN1E', 2), ('1E1S', 2), ('S41E', 2), ('E1E1', 2), ('1SSS', 2), ('SSS3', 2), ('NNE3', 2), ('3N4E', 2), ('3114', 2), ('14NN', 2), ('NENS', 2), ('411E', 2), ('11E1', 2), ('1E1N', 2), ('EE3N', 2), ('N332', 2), ('332S', 2), ('2S3E', 2), ('NNEE', 2), ('EE23', 2), ('N2SN', 2), ('EEE3', 2), ('2NEN', 2), ('NS22', 2), ('N411', 2), ('411S', 2), ('11SS', 2), ('SEEE', 2), ('EE1E', 2), ('EE11', 2), ('E11S', 2), ('11S2', 2), ('S2EE', 2), ('SE2S', 2), ('3S22', 2), ('2EN1', 2), ('N11E', 2), ('14N

## `common_pattern_mining` Function

This function processes frequent patterns to identify and group common patterns along with their associated identifiers. It generates modified lists of patterns with additional elements (unique identifiers), aggregates those lists, and finally identifies and retains common patterns with a specified frequency.

### Input:
- `frequent_patterns` (list of lists): A list containing frequent patterns, where each pattern is represented as a list of tuples. Each tuple contains a pattern element and its occurrence count.

### Output:
Returns a tuple:
- A list of tuples, each containing a common pattern and a list of associated identifiers.
- A flattened list containing all modified pattern-element-identifier tuples.


In [47]:
def common_pattern_mining(frequent_patterns):
    index = 0
    modified_lists = []
    for frequent_pattern in frequent_patterns:
        original_list = frequent_pattern
        extra_element = resampled_data['id'].unique()[index]

        modified_lists.append([(item[0], extra_element) for item in original_list])
        index += 1
    print(len(modified_lists))
    #     print(modified_list)
    flattened_list = [item for sublist in modified_lists for item in sublist]

    grouped_data = {}

    for key, value in flattened_list:
        if key in grouped_data:
            grouped_data[key].append(value)
        else:
            grouped_data[key] = [value]

    result = [(key, values) for key, values in grouped_data.items() if len(values) >= 2]

    return result, flattened_list
    
common_patterns, pattern_list = common_pattern_mining(frequent_patterns)
common_patterns

14


[('34NN', ['Daniel', 'Drogon']),
 ('1144', ['Daniel', 'Viserion_2']),
 ('411S', ['Daniel', 'Magnolia']),
 ('11SS', ['Daniel', 'Magnolia']),
 ('EE11', ['Daniel', 'Magnolia']),
 ('S2EE', ['Daniel', 'Magnolia', 'Jessy']),
 ('3S22', ['Daniel', 'John Snow']),
 ('N11E', ['Daniel', 'Jessy']),
 ('1NN1', ['Daniel', 'Jessy', 'Viserion_2']),
 ('2EE4', ['Daniel', 'Magnolia']),
 ('S33S', ['Magnolia', 'Jessy', 'Viserion_2']),
 ('111E', ['Magnolia', 'Jessy']),
 ('11EE', ['Magnolia', 'Jessy']),
 ('N4SS', ['Jessy', 'Drogon']),
 ('E333', ['Jessy', 'Drogon']),
 ('NES3', ['Jessy', 'Drogon']),
 ('3S23', ['Drogon', 'John Snow']),
 ('34N1', ['Drogon', 'Olenna Tyrell']),
 ('332E', ['John Snow', 'Viserion_2']),
 ('NNS2', ['Samwell Tarly', 'Daenerys'])]

Finding the rows/indeces where the particular patterns were found 

In [48]:
import re

def find_all_pattern_rows(smalldf, patterns):
    all_start_positions = []  # Initialize a list to store all start positions

    pattern_string = ''.join(smalldf['direction'])
    
    for pattern in patterns:
        occurrences = re.finditer(pattern, pattern_string)
        start_positions = [match.start() for match in occurrences]
        all_start_positions.extend(start_positions)  # Add start positions to the list
        
    return all_start_positions  # Return the list of all start positions


combined_df = pd.concat(smalldfs, ignore_index=True)
filtered_rows = combined_df[combined_df['id'].isin(['Daniel', 'Magnolia', 'Jessy'])]

# rows = (find_all_pattern_rows(filtered_rows, ['S2EE']))
rows = (find_all_pattern_rows(filtered_rows, ['34NN']))

result_rows = []

for index in rows:
    if index + 3 < len(combined_df):
        result_rows.append(combined_df.iloc[index:index+4])
    else:
        result_rows.append(combined_df.iloc[index:])

result = pd.concat(result_rows)
result.reset_index(inplace = True)
result.drop(['index'], axis = 1, inplace = True)
result

,ts,lat,lon,id,color,prev_lat,prev_lon,distance,direction,Segment
0,2022-11-16 12:00:00,9.159576,-79.837913,Daniel,#8766EB,9.160940,-79.837188,170.621740,3,D
1,2022-11-16 13:00:00,9.159661,-79.838024,Daniel,#8766EB,9.159576,-79.837913,15.366753,4,D
2,2022-11-16 14:00:00,9.160352,-79.837851,Daniel,#8766EB,9.159661,-79.838024,78.721281,N,D
3,2022-11-16 15:00:00,9.161043,-79.838017,Daniel,#8766EB,9.160352,-79.837851,78.582639,N,D
4,2022-11-20 09:00:00,9.155785,-79.838357,Daniel,#8766EB,9.158521,-79.837188,328.755908,3,M
5,2022-11-20 10:00:00,9.157456,-79.840714,Daniel,#8766EB,9.155785,-79.838357,318.130397,4,M
6,2022-11-20 11:00:00,9.159170,-79.840736,Daniel,#8766EB,9.157456,-79.840714,189.632969,N,D
7,2022-11-20 12:00:00,9.159909,-79.840502,Daniel,#8766EB,9.159170,-79.840736,85.650441,N,D
8,2022-12-14 13:00:00,9.158845,-79.840317,Daniel,#8766EB,9.158871,-79.840306,3.137213,3,D
9,2022-12-14 14:00:00,9.158944,-79.840482,Daniel,#8766EB,9.158845,-79.840317,21.142534,4,D


In [49]:
plotting(result, 'Many patterns')

In [9]:
import re
def translate_time(input_str):
    time_dict = {
        'D': 'Daytime',
        'E': 'Evening',
        'M': 'Morning'
    }
    
    translation = ''
    for char in input_str:
        if char in time_dict:
            translation += f' to {time_dict[char]}'
        else:
            translation += char
    
    return translationx

def get_time(search_string,tdf):
    target_string=''.join(tdf['direction'])
    #print(search_string)
    regex_pattern = re.escape(search_string).replace(r'\ ', r'\s*')
    # Use the regular expression to search the target string
    match = re.search(regex_pattern, target_string)
    if match ==None:
        return 'Unknown time'
    return translate_time(''.join(tdf.iloc[match.start():match.start()+len(search_string)-1,-1][:2]))

In [10]:
from collections import defaultdict
import Levenshtein as lev

def cluster_patterns(patterns_list, threshold=2):
    clusters = defaultdict(list)
    for i, (pattern, freq) in enumerate(patterns_list):
        found_cluster = False
        for cluster_pattern in clusters:
            if lev.distance(pattern, cluster_pattern) <= threshold:
                clusters[cluster_pattern].append((pattern, freq))
                found_cluster = True
                break
        if not found_cluster:
            clusters[pattern].append((pattern, freq))
    
    return clusters.values()

# Example patterns list
# patterns_list = [
#     ('E443', 4), ('E443S', 3), ('3NEE', 3), ('EEEN', 3), ('2E44', 3), ('443S', 3), 
#     ('2E443S', 2), ('E2444', 2), ('1333N', 2), ('3NEE4', 2), ('44N22', 2), ('2E443', 2), 
#     ('13NEE', 2), ('S214', 2), ('4ESS', 2), ('S31N', 2), ('E244', 2), ('2444', 2), ('1333', 2), 
#     ('333N', 2), ('33NE', 2), ('S3SE', 2), ('4NE1', 2), ('3N1N', 2), ('1N23', 2), ('NEE4', 2), 
#     ('EE44', 2), ('44N2', 2), ('4N22', 2), ('N144', 2), ('E11N', 2), ('SEEE', 2), ('43S2', 2), 
#     ('3S2E', 2), ('11NS', 2), ('S41N', 2), ('NE1S', 2), ('1EE4', 2), ('E4SS', 2), ('4SS3', 2), 
#     ('NNSE', 2), ('13NE', 2), ('EN23', 2), ('EN3S', 2)
# ]
patterns_list = frequent_patterns[0]

# Clustering patterns with edit distance threshold of 2
clusters = cluster_patterns(patterns_list, threshold=2)

# Sorting clusters by the sum of the frequency of patterns within the cluster in descending order
sorted_clusters = sorted(clusters, key=lambda cluster: sum(freq for _, freq in cluster), reverse=True)

# Printing the top 5 clusters
top_5_clusters = sorted_clusters[:5]
x=[]
clust=0
for clus in top_5_clusters:
    clust=clust+1
    for pattern in clus[:5]:
        x.append({'Movement Pattern':f'{clust}','direction':pattern[0],'frequency':pattern[1],'time':get_time(pattern[0],smalldf) })
     

d = pd.DataFrame(x)

In [21]:
rows = (find_all_pattern_rows(filtered_rows, ['EEEE', 'EEE1', 'EEEE1', '2EEEE']))


In [22]:
plotting(rows, )

[409]

I apologize for the misunderstanding. Let's identify the common parts within each cluster and use that information to generate a story for each cluster, including details about the time of occurrence:

**Cluster 1:**
Common Part: 443
Explanation: In this cluster, the common movement component is '443'. The monkey frequently moves towards the North West during the transition from Daytime to Evening. It explores this direction consistently, indicating a preference for this path during the evening hours.

**Cluster 2:**
Common Part: 3NEE
Explanation: The common movement component in this cluster is '3NEE'. The monkey often moves towards the South West direction during the transition from Daytime to Evening. This specific movement suggests a tendency to explore the South West quadrant of the jungle in the early evening.

**Cluster 3:**
Common Part: 4
Explanation: Within this cluster, the common movement component is '4'. The monkey's movements often involve heading towards the North West direction, occurring primarily during the evening hours. This movement pattern may indicate the monkey's interest in this specific direction during the evening.

**Cluster 4:**
Common Part: E
Explanation: In this cluster, the common movement component is 'E'. The monkey's movements frequently involve transitioning from Daytime to Morning, reflecting its active exploration during the morning hours. This suggests an adaptive behavior to make the most of the daytime.

**Cluster 5:**
Common Part: 3
Explanation: Within this cluster, the common movement component is '3'. The monkey's movements in this group lack specific time information but often involve moving towards the South West direction. This movement could be exploratory, with no distinct time pattern.

In summary, each cluster exhibits specific common movement components that the monkey tends to follow. These patterns provide insights into the monkey's preferred directions and the timing of its activities during different parts of the day. The common parts highlight the monkey's adaptive strategies as it explores the jungle's diverse regions, tailoring its movements to the surrounding environment and the prevailing time of day.